In [1]:
import os, sys, re, json, time
from importlib import reload
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import *
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Read the All the News Data

In [2]:
articles_1=pd.read_csv('articles1.csv')
articles_2=pd.read_csv('articles2.csv')
articles_3=pd.read_csv('articles3.csv')

In [3]:
all_articles=pd.concat([articles_1,articles_2,articles_3],ignore_index=True)

In [4]:
all_articles.shape

(142570, 10)

In [10]:
all_articles.columns

Index(['Unnamed: 0', 'id', 'title', 'publication', 'author', 'date', 'year',
       'month', 'url', 'content'],
      dtype='object')

In [17]:
np.unique(all_articles['publication'].tolist())

array(['Atlantic', 'Breitbart', 'Business Insider', 'Buzzfeed News',
       'CNN', 'Fox News', 'Guardian', 'NPR', 'National Review',
       'New York Post', 'New York Times', 'Reuters',
       'Talking Points Memo', 'Vox', 'Washington Post'], dtype='<U19')

# Create the Training, Dev, and Test Sets

In [120]:
breitbart=all_articles.loc[all_articles['publication']=='Breitbart'].iloc[0:1000]
nat_review=all_articles.loc[all_articles['publication']=='National Review'].iloc[0:1000]
cnn=all_articles.loc[all_articles['publication']=='CNN'].iloc[0:1000]
guardian=all_articles.loc[all_articles['publication']=='Guardian'].iloc[0:1000]
reuters=all_articles.loc[all_articles['publication']=='Reuters'].iloc[0:1000]
bi=all_articles.loc[all_articles['publication']=='Business Insider'].iloc[0:1000]
full=pd.concat([breitbart,nat_review,cnn,guardian,reuters,bi],ignore_index=True)
shuffle=np.random.permutation(np.arange(full.shape[0]))
values_dict={'CNN': 0,'Guardian':0,'Reuters':1,'Business Insider':1,'Breitbart':2,'National Review':2}
full['label']=full['publication'].map(values_dict)

X,y=full['content'].iloc[shuffle].as_matrix(),full['label'].iloc[shuffle].as_matrix()
train_length=int(shuffle.shape[0]*0.8)
X_train,y_train=X[:train_length],y[:train_length]
X_dev,y_dev=X[train_length:],y[train_length:]

print(X_train.shape)
print(y_train.shape)
print(X_dev.shape)
print(y_dev.shape)

(4800,)
(4800,)
(1200,)
(1200,)


# Fit Count Vectorizer

In [129]:
vec=CountVectorizer(ngram_range=(2,2))
#vec=TfidfVectorizer(ngram_range=(2,2))
feature_mat=vec.fit_transform(X_train)
dev_mat=vec.transform(X_dev)

### train logistic regression model

In [143]:
from sklearn import metrics
Reg=LogisticRegression(penalty='l2')
Reg.fit(feature_mat,y_train)
weights=Reg.coef_
features=vec.get_feature_names()
preds=Reg.predict(dev_mat)
f1=metrics.f1_score(y_dev,preds,average='micro')
print(f1)

0.835


In [144]:
print(weights.shape)

(3, 1027550)


# Load IBC Data

## Read the data

In [131]:
ibc_frame=pd.read_csv('full_ibc_sentences.csv')
ibc_values_dict={'Liberal':0,'Neutral':1,'Conservative':2}
ibc_frame['class']=ibc_frame['label'].map(ibc_values_dict)
sentences=ibc_frame['sentence'].as_matrix()
labels=ibc_frame['class'].as_matrix()

## Score Model Above on IBC Data - Just to see how well it works

In [137]:
test_mat=vec.transform(sentences)

In [138]:
preds=Reg.predict(test_mat)
f1=metrics.f1_score(labels,preds,average='micro')
print(f1)

0.16689782709200185
